In [1]:
import pandas as pd
import numpy as np

### data loading

In [2]:
database_path = "/Users/jinmr2/Dropbox/Code/data/ToppGene_Data/"
path = "/Users/jinmr2/Dropbox/Code/data/toppcell_test/"

In [3]:
df_deg = pd.read_csv(path + "deg_shredplan.txt", sep = "\t", header = 0, index_col = 0)

/Users/jinmr2/opt/miniconda3/envs/aipy/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df_CD14 = df_deg.loc[df_deg["Status"] == "CD14+ Monocytes", :].head(200)
df_CD14.head()

,Status,scores,logfoldchanges,pvals,pvals_adj,pts,pts_rest,reference_group,shred_plan
names,,,,,,,,,
RPL32,CD14+ Monocytes,93.365440,1.821397,0.0,0.0,0.947075,0.973766,NaN,cell
RPS18,CD14+ Monocytes,92.939728,1.931638,0.0,0.0,0.937976,0.980091,NaN,cell
PABPC1,CD14+ Monocytes,92.322075,3.812964,0.0,0.0,0.742061,0.825021,NaN,cell
RPS14,CD14+ Monocytes,92.026360,1.774112,0.0,0.0,0.932962,0.980662,NaN,cell
RPS6,CD14+ Monocytes,90.927338,2.077077,0.0,0.0,0.890251,0.981906,NaN,cell


In [5]:
df_cellcompartment = pd.read_csv(database_path + "GeneOntologyCellularComponent.txt", sep = "\t", header = 0, index_col = 0)

In [6]:
df_cellcompartment.head()

,symbol,concept_id,concept_name,concept_origin,url
gene_id,,,,,
387712,ENO4,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
2023,ENO1,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
2026,ENO2,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
2027,ENO3,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
1161,ERCC8,GO:0000109,nucleotide-excision repair complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...


In [9]:
df_CD14.shape

(200, 9)

### get a pool

In [10]:
test = df_cellcompartment.copy()
test.set_index("symbol", inplace = True)

In [22]:
test.head()

,concept_id,concept_name,concept_origin,url
symbol,,,,
ENO4,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
ENO1,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
ENO2,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
ENO3,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
ERCC8,GO:0000109,nucleotide-excision repair complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...


In [24]:
pool = np.unique(test.loc[gene_list,"concept_name"])
len(pool)

107

### enrichment for each concept

#### preparation of ranked file

In [16]:
df_CD14["gene"] = df_CD14.index.values
df_CD14 = df_CD14[["gene", "scores"]]

In [17]:
df_CD14.head(2)

,gene,scores
names,,
RPL32,RPL32,93.365440
RPS18,RPS18,92.939728


#### make the table into a dictionary

In [27]:
df_cellcompartment.head(2)

,symbol,concept_id,concept_name,concept_origin,url
gene_id,,,,,
387712,ENO4,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
2023,ENO1,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...


In [30]:
enrich_dict = {}
for gene_set in np.unique(df_cellcompartment["concept_name"]):
    genes = df_cellcompartment.loc[df_cellcompartment["concept_name"] == gene_set, "symbol"]
    enrich_dict[gene_set] = list(genes)

#### do exercise on single pre-rank enrichment

In [23]:
gene_set_test = df_cellcompartment.loc[df_cellcompartment["concept_id"]== "GO:0000118",:]

In [25]:
gene_set_test.head()

,symbol,concept_id,concept_name,concept_origin,url
gene_id,,,,,
55809,TRERF1,GO:0000118,histone deacetylase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
9219,MTA2,GO:0000118,histone deacetylase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
10629,TAF6L,GO:0000118,histone deacetylase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
9734,HDAC9,GO:0000118,histone deacetylase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
8841,HDAC3,GO:0000118,histone deacetylase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...


In [ ]:
dict(zip(gene_set_test.index.values, ))

In [41]:
pre_res = gp.prerank(rnk = df_CD14,
                    gene_sets = enrich_dict,
                    processes = 4, permutation_num = 100,
                    min_size = 5, max_size = 1500)

In [42]:
pre_res.res2d.sort_index().head()

,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
anchoring junction,0.592427,1.928748,0.000000,0.000000,880,50,RPS18;PABPC1;RPS14;RPL13A;RPL7;RPL31;RPS4X;RPS...,RPS18;PABPC1;RPS14;RPL13A;RPL7;RPL31;RPS4X;RPS...
asymmetric synapse,0.641728,1.849722,0.000000,0.000402,439,20,RPS18;RPS14;RPS27;RPL7;RPS13;RPS3;RPL30;RPS25;...,RPS18;RPS14;RPS27;RPL7;RPS13;RPS3;RPL30;RPS25;...
bounding membrane of organelle,0.202797,0.591388,0.936842,0.979920,2203,21,RPS27A;SELL;CD3D;RPS28;IL7R;RPL27;EEF1A1;GIMAP...,RPS27A;SELL;CD3D;RPS28;IL7R;RPL27;EEF1A1;GIMAP...
cell junction,0.621495,2.008112,0.000000,0.000000,2317,74,RPS18;PABPC1;RPS14;RPS27;RPL13A;RPL10;RPL7;RPL...,RPS18;PABPC1;RPS14;RPS27;RPL13A;RPL10;RPL7;RPL...
cell periphery,0.246318,0.806315,0.800000,0.923408,9698,90,PABPC1;RPS6;RPL13;RPL34;RPS12;RPL21;RPL10;GIMA...,PABPC1;RPS6;RPL13;RPL34;RPS12;RPL21;RPL10;GIMA...


In [46]:
result = pre_res.res2d
result = result.sort_values(["fdr"], ascending = True)
result.head()

,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
anchoring junction,0.592427,1.928748,0.0,0.0,880,50,RPS18;PABPC1;RPS14;RPL13A;RPL7;RPL31;RPS4X;RPS...,RPS18;PABPC1;RPS14;RPL13A;RPL7;RPL31;RPS4X;RPS...
intrinsic component of membrane,-0.187722,inf,NaN,0.0,5584,37,LTB;CCR7;SELL;CD3D;IL7R;LEPROTL1;GIMAP5;TRAT1;...,SELL;CD3D;IL7R;LEPROTL1;GIMAP5;TRAT1;CD7;PIK3I...
intracellular vesicle,-0.219128,inf,NaN,0.0,2561,27,RPS27A;SELL;CD3D;ARHGDIB;IL7R;LEPROTL1;TMSB4X;...,SELL;CD3D;ARHGDIB;IL7R;LEPROTL1;TMSB4X;TPT1;EE...
cytoplasmic vesicle,-0.219128,inf,NaN,0.0,2555,27,RPS27A;SELL;CD3D;ARHGDIB;IL7R;LEPROTL1;TMSB4X;...,SELL;CD3D;ARHGDIB;IL7R;LEPROTL1;TMSB4X;TPT1;EE...
synapse,0.648706,2.064668,0.0,0.0,1582,57,RPS18;PABPC1;RPS14;RPS27;RPL10;RPL7;RPL31;RPL1...,RPS18;PABPC1;RPS14;RPS27;RPL10;RPL7;RPL31;RPL1...


#### do the prerank enrichment

In [12]:
import gseapy as gp

In [ ]:
def 